In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np

In [ ]:
prot = md.load_pdb('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU_cv.pdb')
topology = prot.topology

g1 = [atom.index for atom in topology.chain(0).atoms]
g2 = [atom.index for atom in topology.chain(1).atoms]
g3 = [atom.index for atom in topology.chain(2).atoms]
g4 = [atom.index for atom in topology.chain(3).atoms]

In [ ]:
prmtop = AmberPrmtopFile('apart-lastframe.prmtop')
inpcrd = AmberInpcrdFile('apart-lastframe.inpcrd')
pdb = PDBFile('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU_cv.pdb')

system = prmtop.createSystem(nonbondedMethod=NoCutoff, constraints=HBonds, hydrogenMass=1.5*amu, 
                             implicitSolvent=OBC2)

expr = 'distance(g1,g2)+distance(g1,g3)+distance(g1,g4)+distance(g2,g3)+distance(g2,g4)+distance(g3,g4)'


dist_force_1 = openmm.CustomCentroidBondForce(4, expr)
dist_force_1.addGroup(g1)
dist_force_1.addGroup(g2)
dist_force_1.addGroup(g3)
dist_force_1.addGroup(g4)
dist_force_1.addBond([0,1,2,3])
#dist_force.setForceGroup(1)
#do not add force to the system manually, it will get added in the metadynamics step as a CV force
dist_bias_1 = BiasVariable(dist_force_1, 1, 100, 0.5, False)


meta = Metadynamics(system, [dist_bias_1], 298.15*kelvin, 5.0, 1*kilojoule_per_mole, 1)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.004*picoseconds)

platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)

simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('test-output.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
meta.step(simulation, 100000)
